[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/viincci/Viincci-RAG/blob/main/viincci_rag/examples/test_examples.ipynb)

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

Run this notebook in **Google Colab** (link above) or locally with Jupyter.

**Features:**
- Backwards-compatible imports (viincci_rag → V4 fallback)
- SerpAPI mock to avoid using credits while testing
- Multiple domain examples (botany, literature, carpentry)
- Safe fallback generation when library unavailable

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

This notebook provides a small, safe set of example cells to:

- Prefer imports from `viincci_rag` (new layout) and fall back to the legacy `V4` package when needed (backwards compatible).
- Demonstrate a SerpAPI mock so you can run examples without consuming real SerpAPI credits (useful for the free 250 credits/month constraint).
- Provide tiny generation examples across several domains using either the real library (if installed) or a harmless fallback that stitches mock research results together.

Notes: Run cells sequentially. If you have the real library installed and a valid SerpAPI key configured, the notebook will attempt to use it; otherwise it uses safe mocks.

## Installation (Optional - uncomment if running in fresh Colab)

If you're running in Google Colab and don't have the library installed:

In [ ]:
# Uncomment the line below to install from GitHub if needed
# !pip install -q git+https://github.com/viincci/Viincci-RAG.git

## 1. Setup: Imports and Mock SerpAPI

In [ ]:
# Ensure workspace package is importable (add repo root to sys.path)
import sys
import os

# Try to add workspace root to sys.path if running from notebook
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in dir() else os.getcwd()
# Navigate up from examples/ -> viincci_rag/ -> repo root
repo_root = os.path.abspath(os.path.join(notebook_dir, '../..'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    print(f"✓ Added repo root to sys.path: {repo_root}")

# Backwards-compatible imports: prefer viincci_rag, fallback to V4 (legacy)
import importlib

def try_import():
    """Try to import core modules from viincci_rag (primary), fall back to V4 (legacy)."""
    # Try new package layout first
    try:
        from viincci_rag.core.config import ConfigManager
        from viincci_rag.core.spider import UniversalResearchSpider
        from viincci_rag.core.rag_system import RAGSystem
        from viincci_rag.core.article_generator import UniversalArticleGenerator
        print('✅ Imported core modules from viincci_rag (new layout)')
        return {
            'ConfigManager': ConfigManager,
            'UniversalResearchSpider': UniversalResearchSpider,
            'RAGSystem': RAGSystem,
            'UniversalArticleGenerator': UniversalArticleGenerator
        }
    except Exception as e_new:
        # Try legacy package name used in older releases
        try:
            from V4 import ConfigManager, UniversalResearchSpider, RAGSystem, UniversalArticleGenerator
            print('✅ Imported core modules from V4 (legacy layout - backwards compatible)')
            return {
                'ConfigManager': ConfigManager,
                'UniversalResearchSpider': UniversalResearchSpider,
                'RAGSystem': RAGSystem,
                'UniversalArticleGenerator': UniversalArticleGenerator
            }
        except Exception as e_legacy:
            print('⚠️  Could not import core modules from viincci_rag or V4. Continuing with safe mocks.')
            return {}

viincci_modules = try_import()

# Expose names into notebook globals if real modules were found
if viincci_modules:
    globals().update(viincci_modules)

In [ ]:
# A tiny SerpAPI mock — returns a list of dicts with text and metadata
class MockSerpAPI:
    def __init__(self, key=None):
        self.key = key
    
    def search(self, query, num_results=3):
        """Return deterministic mock results based on the query string."""
        mock_results = []
        for i in range(num_results):
            mock_results.append({
                'text': f"Mock result {i+1} for '{query}'. This is a simulated search result.",
                'title': f"Mock Title {i+1} - {query.split()[0]}",
                'link': f"http://mock.example.com/{query.replace(' ', '-')}-{i+1}"
            })
        return mock_results

def get_serp_api_client():
    """Prefer real SerpAPI client if key and package available, else mock."""
    key = os.environ.get('SERP_API_KEY')
    if key:
        try:
            # Lazy import of serpapi client if available
            from serpapi import GoogleSearch
            print("✅ Using real SerpAPI client.")
            return GoogleSearch({'api_key': key})
        except ImportError:
            print("⚠️  SerpAPI client not installed, falling back to mock.")
            return MockSerpAPI(key)
        except Exception as e:
            print(f"⚠️  Error initializing SerpAPI client: {e}, falling back to mock.")
            return MockSerpAPI(key)
    else:
        print("ℹ️  SERP_API_KEY not found, using MockSerpAPI.")
        return MockSerpAPI()

serp_api = get_serp_api_client()

## 2. Helper Functions

In [ ]:
def normalize_search_results(results):
    """Convert search results into the shape expected by RAG systems."""
    normalized = []
    for r in results:
        normalized.append({
            'text': r.get('text', ''),
            'metadata': {
                'title': r.get('title'),
                'link': r.get('link'),
                **r.get('metadata', {})
            }
        })
    return normalized

# Example queries across different domains
queries = {
    'literature': 'Edgar Allan Poe poetry style themes Gothic',
    'botany': 'photosynthesis process details plant cells',
    'carpentry': 'wood joinery techniques types dovetail mortise tenon'
}

print(f"✅ Setup complete. Queries ready: {list(queries.keys())}")

## 3. Example: Mock Search Results

In [ ]:
# Run a mock search and display normalized results
query = queries['literature']
print(f"\n🔍 Searching: {query}")
results = serp_api.search(query, num_results=3)
normalized = normalize_search_results(results)

print(f"\n✅ Found {len(normalized)} results:")
for i, result in enumerate(normalized, 1):
    print(f"\n  [{i}] {result['metadata']['title']}")
    print(f"      Link: {result['metadata']['link']}")
    print(f"      Text: {result['text'][:100]}...")

## 4. Example: Using Real Modules (if available)

In [ ]:
# If real modules were imported successfully, you can use them here
if viincci_modules:
    print("✅ Real viincci_rag modules are available!")
    print(f"   Available: {list(viincci_modules.keys())}")
    # Example: 
    # config = ConfigManager()
    # spider = UniversalResearchSpider(config)
    # rag = RAGSystem(config)
    # article = UniversalArticleGenerator(config)
else:
    print("⚠️  Real modules not available. Using mock functions for demonstration.")

## 5. Additional Examples (Botany & Carpentry)

In [ ]:
# Example 2: Botany query
query = queries['botany']
print(f"\n🔍 Searching: {query}")
results = serp_api.search(query, num_results=2)
for i, result in enumerate(results, 1):
    print(f"\n  [{i}] {result['title']}")
    print(f"      {result['text'][:80]}...")

In [ ]:
# Example 3: Carpentry query
query = queries['carpentry']
print(f"\n🔍 Searching: {query}")
results = serp_api.search(query, num_results=2)
for i, result in enumerate(results, 1):
    print(f"\n  [{i}] {result['title']}")
    print(f"      {result['text'][:80]}...")

## Notes

- This notebook prioritizes `viincci_rag` imports but gracefully falls back to `V4` for backwards compatibility
- All searches use mock data by default (no API credits consumed)
- To use real SerpAPI, set the `SERP_API_KEY` environment variable and ensure `serpapi` is installed
- For real article generation, ensure your environment has the actual viincci_rag package installed

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/viincci/Viincci-RAG/blob/main/viincci_rag/examples/test_examples.ipynb)

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

Run this notebook in **Google Colab** (link above) or locally with Jupyter.

**Features:**
- Backwards-compatible imports (viincci_rag → V4 fallback)
- SerpAPI mock to avoid using credits while testing
- Multiple domain examples (botany, literature, carpentry)
- Safe fallback generation when library unavailable

# Viincci-RAG — Minimal Examples and Safe SerpAPI Mock

This notebook provides a small, safe set of example cells to:

- Prefer imports from `viincci_rag` (new layout) and fall back to the legacy `V4` package when needed (backwards compatible).
- Demonstrate a SerpAPI mock so you can run examples without consuming real SerpAPI credits (useful for the free 250 credits/month constraint).
- Provide tiny generation examples across several domains using either the real library (if installed) or a harmless fallback that stitches mock research results together.

Notes: Run cells sequentially. If you have the real library installed and a valid SerpAPI key configured, the notebook will attempt to use it; otherwise it uses safe mocks.

In [ ]:
# Backwards-compatible imports: prefer viincci_rag, fallback to V4 (legacy)
import importlib

,
def try_import():
    globals_local = {}
    # Try new package layout first
    try:
        # attempt to import the new package layout (viincci_rag)
        from viincci_rag.core.config import ConfigManager
        from viincci_rag.core.spider import UniversalResearchSpider
        from viincci_rag.core.rag_system import RAGSystem
        from viincci_rag.core.article_generator import UniversalArticleGenerator
        print('Imported core modules from viincci_rag (new layout)')
        return {
            'ConfigManager': ConfigManager,
            'UniversalResearchSpider': UniversalResearchSpider,
            'RAGSystem': RAGSystem,
            'UniversalArticleGenerator': UniversalArticleGenerator
        }
    except Exception as e_new:
        # Try legacy package name used in older releases
        try:
            from V4 import ConfigManager, UniversalResearchSpider, RAGSystem, UniversalArticleGenerator
            print('Imported core modules from V4 (legacy layout)')
            return {
                'ConfigManager': ConfigManager,
                'UniversalResearchSpider': UniversalResearchSpider,
                'RAGSystem': RAGSystem,
                'UniversalArticleGenerator': UniversalArticleGenerator
            }
        except Exception as e_legacy:
            print('Could not import core modules from viincci_rag or V4. Continuing with safe mocks.')
            return {}
viincci_modules = try_import()
# Expose names into notebook globals if real modules were found
if viincci_modules:
    globals().update(viincci_modules)
: 
,
: {
: 

: [
,
,

: 
,
: {
: 

: [
,
# A tiny SerpAPI mock — returns a list of 
 dicts with text and metadata
class MockSerpAPI:
    def __init__(self, key=None):
        self.key = key
    def search(self, query, num_results=3):
        # Deterministic mock results based on the query string
        return [
    # Prefer a real SerpAPI client if the key is present and the package is installed
    key = os.environ.get('SERP_API_KEY')
    if key:
        try:
            # lazy import of serpapi client if available in the environment
            from serpapi import GoogleSearch
            return GoogleSearch({'api_key': key})
        except Exception:
,
    else:
,

In [ ]:
# Small helper to convert search results into the shape expected by RAG systems in this repo
def normalize_search_results(results):
    normalized = []
    for r in results:
        normalized.append({'text': r.get('text', ''), 'metadata': {'title': r.get('title'), **r.get('metadata', {})}})
,
# Example: run safe searches and produce short generated outputs using either the real generators or a fallback
queries = {
    'literature': 'Edgar Allan Poe poetry style themes Gothic',